In [1116]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path 
import sys 
import os 
import logging
import seaborn as sns
from decimal import Decimal, ROUND_DOWN

from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, GroupShuffleSplit
from sklearn.metrics import make_scorer ,root_mean_squared_error

from collections import Counter
from sklearn.preprocessing import FunctionTransformer, StandardScaler

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold,  cross_validate
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor
from sklearn.svm import SVR

from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from skrub import TableVectorizer


from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report, confusion_matrix

sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict, audit, plot, save


%load_ext autoreload
%autoreload 2

#shap.initjs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1117]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived  = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived  = '/Users/alina/Desktop/MIT/code/data/output/derived_data'

info_path = Path(data_root, "files")
#os.listdir(data_derived)


In [1118]:
################## PREDICTORS ####################
pred = pd.read_csv(Path(data_derived, 'mta_data_clean.csv')).drop(columns = 'Unnamed: 0')
out = pd.read_csv(Path(data_derived, 'out_clean_all_raters.csv')).drop(columns = 'Unnamed: 0')

rater_out = 'm'
rater_pred = 't' # if set to None use all raters 

thr_drop_missing = 20 # if changed to lower number, some cols will be dropped, 50 iwas threshold in preprcessing (wont change anything)


In [1119]:

if rater_pred is not None:
    col_pred = [ col for col in pred.columns if col.endswith(rater_pred)]
    col_pred.append("src_subject_id")
    col_pred.append('trtname')
    pred = pred[col_pred]
    
    
 ################ OUTCOMES ##################   

col_out_rater = [col for col in out if col.endswith(rater_out)]

out_rater = out[np.concatenate((['src_subject_id'], col_out_rater))]

for col in col_out_rater:
    print(col, out_rater[col].dropna().shape)

out_rater = out_rater.rename(columns={col: f"{col}_out" for col in out_rater.columns if col != 'src_subject_id'})


################ MERGE ##############
data = pd.merge(out_rater, pred, how = 'left', on = 'src_subject_id')
data = audit.remove_cols(data, thr_drop_missing=thr_drop_missing) # adjust if desired, as loaded thr was 50 

y_cols = [col for col in data.columns if col.endswith('out')]
print('possible outcomes : ', y_cols)
y_col = y_cols[2] # extrcat single outcome to predict 
print('To predict : ', y_col)

X_cols = [col for col in data.columns if col not in y_cols]
data = data.dropna(subset=[y_col])

y = np.array(data[y_col])
df_X = data[X_cols].drop(columns='src_subject_id')

snap_snainatt_m (502,)
snap_snahypat_m (503,)
snap_snaoddt_m (503,)
ssrs_sspintt_m (497,)
ssrs_ssptosst_m (496,)
pcrc_pcrcpax_m (520,)
pcrc_pcrcprx_m (521,)
Index([], dtype='object')
Removing empty columns ..  N =  0
Index([], dtype='object')
(559, 147)
Removing constant columns .. N =  0
Index([], dtype='object')
(559, 147)
Removing known and raw columns..  N =  : 0
[]
(559, 147)
Removing above threshold empty columns.. N =  : 0
set()
(559, 147)
possible outcomes :  ['snap_snainatt_m_out', 'snap_snahypat_m_out', 'snap_snaoddt_m_out', 'ssrs_sspintt_m_out', 'ssrs_ssptosst_m_out', 'pcrc_pcrcpax_m_out', 'pcrc_pcrcprx_m_out']
To predict :  snap_snaoddt_m_out


In [1120]:

col_names_data = list(data.columns)

ord_vars, num_vars, cat_vars = [], [], []

types_df = pd.read_excel(Path(data_derived, 'all_vars_description_ML.xlsx'), sheet_name='Sheet1')

for _, row in types_df.iterrows():
    var_name = row[1]  # e.g. variable name in the spreadsheet
    var_type = row[4]  # e.g. "ord" / "num" / "cat"

    # Collect all columns in `data` that contain `var_name`
    var_in_data = [col for col in col_names_data if var_name in col]

    if var_type == "ord":
        ord_vars.append(var_in_data)
    elif var_type == "num":
        num_vars.append(var_in_data)
    elif var_type == "cat":
        cat_vars.append(var_in_data)

# Example: manually add a column named 'trtname' to cat_vars
cat_vars.append(['trtname'])

# Flatten each list-of-lists into a single array
ord_vars = np.concatenate(ord_vars)
cat_vars = np.concatenate(cat_vars)
num_vars = np.concatenate(num_vars)

# Convert them to plain Python strings
ord_vars = [str(col) for col in ord_vars]
cat_vars = [str(col) for col in cat_vars]
num_vars = [str(col) for col in num_vars]

print("Ordinal vars:", ord_vars)
print("Numeric vars:", num_vars)
print("Categorical vars:", cat_vars)

num_vars_in = [str(col) for col in num_vars if not col.endswith("out")] # name of numerical variables present in dataframe X (excludin var names in y)


Ordinal vars: ['snap_sna19_t', 'snap_sna20_t', 'snap_sna21_t', 'snap_sna22_t', 'snap_sna23_t', 'snap_sna24_t', 'snap_sna25_t', 'snap_sna26_t', 'snap_sna27_t', 'snap_sna28_t', 'snap_sna29_t', 'snap_sna30_t', 'snap_sna31_t', 'snap_sna32_t', 'snap_sna33_t', 'snap_sna34_t', 'snap_sna35_t', 'snap_sna36_t', 'snap_sna37_t', 'snap_sna38_t', 'snap_sna39_t', 'snap_snap_adhd_1_t', 'snap_snap_adhd_11_t', 'snap_snap_adhd_12_t', 'snap_snap_adhd_13_t', 'snap_snap_adhd_14_t', 'snap_snap_adhd_15_t', 'snap_snap_adhd_16_t', 'snap_snap_adhd_17_t', 'snap_snap_adhd_18_t', 'snap_snap_adhd_19_t', 'snap_snap_adhd_2_t', 'snap_snap_adhd_3_t', 'snap_snap_adhd_4_t', 'snap_snap_adhd_5_t', 'snap_snap_adhd_6_t', 'snap_snap_adhd_7_t', 'snap_snap_adhd_8_t', 'snap_snap_adhd_9_t', 'snap_snap_adhd_11_t', 'snap_snap_adhd_12_t', 'snap_snap_adhd_13_t', 'snap_snap_adhd_14_t', 'snap_snap_adhd_15_t', 'snap_snap_adhd_16_t', 'snap_snap_adhd_17_t', 'snap_snap_adhd_18_t', 'snap_snap_adhd_19_t', 'ssrs_s7t35_t', 'ssrs_s7t36_t', 'ssrs

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_16220/2697639973.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  var_name = row[1]  # e.g. variable name in the spreadsheet
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_16220/2697639973.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  var_type = row[4]  # e.g. "ord" / "num" / "cat"


In [1121]:
num_set = set(num_vars_in)
ord_set = set(ord_vars)
cat_set = set(cat_vars)

# Pairwise overlaps:
overlap_num_ord = num_set & ord_set
overlap_num_cat = num_set & cat_set
overlap_ord_cat = ord_set & cat_set

print("Overlaps between numeric and ordinal:", overlap_num_ord)
print("Overlaps between numeric and categorical:", overlap_num_cat)
print("Overlaps between ordinal and categorical:", overlap_ord_cat)

# Overlap across all three:
overlap_all_three = num_set & ord_set & cat_set
print("Overlaps across numeric, ordinal and categorical:", overlap_all_three)

Overlaps between numeric and ordinal: set()
Overlaps between numeric and categorical: set()
Overlaps between ordinal and categorical: set()
Overlaps across numeric, ordinal and categorical: set()


In [1122]:

cat_vars_str, cat_vars_num, rest = [], [], []

for col in cat_vars:
    # Get the first non-null value in the column
    val = data[col].dropna().unique()[0]
    
    # Check if it's a (Python or NumPy) string
    if isinstance(val, (str, np.str_)):
        cat_vars_str.append(str(col))  # ensure column name is a Python str
    # Check if it's a (Python or NumPy) float
    elif isinstance(val, (float, np.floating)):
        cat_vars_num.append(str(col))  # ensure column name is a Python str
    else:
        rest.append(str(col))          # store in `rest` for debugging
print(cat_vars_str)
print(cat_vars_num)

['trtname']
[]


In [1123]:
# Define the extensions to check
extensions = ['_m', #mother 
              #'_p'# proffesionals
              '_f', # father,
              '_c',# child,
              '_t'] # teacher 

# Count columns for each extension
extension_counts = {ext: sum(col.endswith(ext) for col in pred.columns) for ext in extensions}

for ext, count in extension_counts.items():
    print(f"Columns ending with '{ext}': {count}")

Columns ending with '_m': 0
Columns ending with '_f': 0
Columns ending with '_c': 0
Columns ending with '_t': 138


In [1124]:
params = { 'tune_hyperparam' : False,
          'temp_size' : 0.3, # Size of 
          'test_size' : 0.3,
          'random_state' : 42, 
          'num_top_features': 10, # number of features to keep after featur selection, change to when it doesnt get better anymore 
          'stratify' : 'trtname',
          'gkf_n_splits' : 5} # in balance in train teat split 


scoring = {
    'r2': 'r2',
    'mse': 'neg_mean_squared_error',
    'mae': 'neg_mean_absolute_error'
}

groups = data['src_subject_id'].values

In [1125]:
# ----------------------------
# Custom Transformers
# ----------------------------

class CorrelationSelector(BaseEstimator, TransformerMixin):
    """
    Selects features based on a correlation threshold.
    Removes features that have a correlation higher than the specified threshold with any other feature.
    """
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.to_drop_ = None
        self.features_to_keep_ = None

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("CorrelationSelector expects input X to be a pandas DataFrame")
        
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.to_drop_ = [column for column in upper.columns if any(upper[column] > self.threshold)]
        self.features_to_keep_ = [column for column in X.columns if column not in self.to_drop_]
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("CorrelationSelector expects input X to be a pandas DataFrame")
        return X[self.features_to_keep_]

    def get_support(self, indices=False):
        if indices:
            return [i for i, col in enumerate(self.features_to_keep_)]
        else:
            return self.features_to_keep_

In [1126]:
df_X.columns.duplicated()

# To list just the duplicated columns:
dup_cols = df_X.columns[df_X.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())

Duplicated column names: []


In [1127]:
df_X.columns.duplicated()

# To list just the duplicated columns:
dup_cols = df_X.columns[df_X.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())
X_select = df_X[list(set(num_vars_in + ord_vars + cat_vars))]
X_select.columns.duplicated()

# To list just the duplicated columns:
dup_cols = X_select.columns[X_select.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())

Duplicated column names: []
Duplicated column names: []


In [1128]:
X_select.columns.duplicated()

# To list just the duplicated columns:
dup_cols = X_select.columns[X_select.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())

Duplicated column names: []


In [1129]:
# Ensure X is a DataFrame
X = X_select.copy()

# Initialize GroupShuffleSplit for initial train-test split
gss_test = GroupShuffleSplit(n_splits=1, test_size=params['test_size'], random_state=params['random_state'])
train_val_idx, test_idx = next(gss_test.split(X, y, groups=groups))

# Split data into training-validation and test sets
X_test, y_test, groups_test = X.iloc[test_idx], y[test_idx], groups[test_idx]
X_train_val, y_train_val, groups_train_val = X.iloc[train_val_idx], y[train_val_idx], groups[train_val_idx]

print(f"Total samples: {X.shape[0]}")
print('TRAIN_VAL ', X_train_val.shape, y_train_val.shape)
print('TEST', X_test.shape, y_test.shape)

Total samples: 503
TRAIN_VAL  (352, 139) (352,)
TEST (151, 139) (151,)


In [1130]:
def to_dataframe(X):
    # Create a DataFrame with generic column names: f0, f1, f2, ...
    n_cols = X.shape[1]
    return pd.DataFrame(X, columns=[f"f{j}" for j in range(n_cols)])
to_df_transformer = FunctionTransformer(func=to_dataframe, validate=False)

In [1131]:
# param_distributions_XGB = {
#     'regressor__n_estimators': [100, 300],     # Number of boosting rounds
#     'regressor__max_depth': [3, 6],           # Maximum tree depth
#     'regressor__learning_rate': [0.05, 0.1],  # Learning rate (shrinkage)
#     'regressor__subsample': [0.8, 1.0],       # Fraction of samples per tree
#     'regressor__colsample_bytree': [0.8, 1.0] # Fraction of features per tree
# }

## Next   Compute 
- simple model results for XG boost , finish for RF
- accuracy as correlation prediction - label ? 
- add cross validation loop  
- feature importances in cross validation 
- split half reliabiliy of feature importances across splits 
- reliability of feature importances across models 

In [1132]:

# param_distributions_XGB = {
#     'regressor__n_estimators': [100, 300],  # Number of trees in the forest
#     'regressor__max_depth': [None, 10],    # Maximum depth of the trees
#     'regressor__max_features': ['sqrt', 'log2'],  # Number of features to consider at each split
#     'regressor__min_samples_split': [2, 10],  # Minimum samples required to split an internal node
#     'regressor__min_samples_leaf': [1, 4]   # Minimum samples required at a leaf node
# }


In [1133]:
regress =  XGBRegressor(random_state=42)


num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('std_scaler', StandardScaler())
])

ord_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('identity', FunctionTransformer(lambda x: x))
])


cat_to_dataframe = FunctionTransformer(
    func=lambda X: pd.DataFrame(X, columns=cat_vars_str),
    validate=False  # prevent extra validation that may alter the input
)

cat_str_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('to_df', cat_to_dataframe),        # conversion step added
     ('ohe', TableVectorizer())
])

cat_num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
    ('ohe', OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_vars_in),
    ('cat_str', cat_str_pipe, cat_vars_str),
    ('cat_num', cat_num_pipe, cat_vars_num),
    ('ord', ord_pipe, ord_vars)
])


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('to_df', to_df_transformer),
    ('correlation_selector', CorrelationSelector(threshold=0.8)),
    ('regressor', regress)
])
param_distributions= {
    'regressor__n_estimators': [100, 300],     # Number of boosting rounds
    'regressor__max_depth': [3, 6],           # Maximum tree depth
    'regressor__learning_rate': [0.05, 0.1],  # Learning rate (shrinkage)
    'regressor__subsample': [0.8, 1.0],       # Fraction of samples per tree
    'regressor__colsample_bytree': [0.8, 1.0] # Fraction of features per tree
}
# Define the parameter grid

# Define the GroupKFold cross-validation
cv = GroupKFold(n_splits=5)

# Define the scorer
scorer = {
    'rmse': make_scorer(root_mean_squared_error, greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'r2': make_scorer(r2_score)
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,  # Number of parameter settings to sample
    scoring=scorer,
    refit='rmse', 
    cv=cv,
    verbose=1,
    random_state=42
)

# Perform the search
random_search.fit(X, y, groups=groups)

# Print best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

#pipeline.fit(X_train_val, y_train_val)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'regressor__subsample': 1.0, 'regressor__n_estimators': 100, 'regressor__max_depth': 3, 'regressor__learning_rate': 0.05, 'regressor__colsample_bytree': 1.0}
Best Score: -5.6026256292534375


In [1134]:
results_df = pd.DataFrame(random_search.cv_results_)
results_df
# Find the row corresponding to the best parameters
best_index = random_search.best_index_

# Extract metrics for the best parameters
best_results = results_df.iloc[best_index]


print("Metrics for Best Parameters:")
print(best_results[['mean_test_rmse', 'mean_test_mae', 'mean_test_r2']])




Metrics for Best Parameters:
mean_test_rmse   -5.602626
mean_test_mae     -4.43264
mean_test_r2      0.014674
Name: 3, dtype: object


In [1135]:
# y_pred = random_search.predict(X_test)
# # Calculate evaluation metrics
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"Mean Squared Error: {mse}")
# print(f"Root Mean Squared Error: {rmse}")
# print(f"Mean Absolute Error: {mae}")
# print(f"R^2 Score: {r2}")


In [1136]:
pipeline.named_steps.keys()

dict_keys(['preprocessor', 'to_df', 'correlation_selector', 'regressor'])

In [1137]:

cross_val_strategy = str(random_search.cv)

params_RF = ['n_estimators', 'criterion', 'max_depth', 'max_features']
params_XGB = [ 'n_estimators', 'learning_rate','max_depth', 'max_leaves', 'tree_method' ]
cols_res = ['Model Name', 'Cross Validation Type', 'Hyperparameters',
       'Mean Squared Error', 'Root Mean Squared Error', 'Mean Absolute Error',
       'R² Score', 'Outcome Variable', 'Input Data',
       'Feature Selection Method', 'Number of Features', 'Threshold Drop Row']

best_model = random_search.best_estimator_.named_steps['regressor']

# Get the number of input features
num_features = best_model.n_features_in_
X_sample = X_train_val 



def truncate(value, decimals):
    value = Decimal(value)
    return float(value.quantize(Decimal(f"1.{'0' * decimals}"), rounding=ROUND_DOWN))


results_df = pd.DataFrame(random_search.cv_results_)
best_index = random_search.best_index_
best_results = results_df.iloc[best_index]

rmse = - truncate(best_results['mean_test_rmse'], 4)
mse =  truncate(best_results['mean_test_rmse'] ** 2, 4)
mae = - truncate(best_results['mean_test_mae'], 4)
r2 = truncate(best_results['mean_test_r2'], 4)


model_name = pipeline.named_steps['regressor'].__class__.__name__

if model_name == 'XGBRegressor': 
    params_to_extract = params_XGB
elif model_name == 'RandomForestRegressor':
    params_to_extract = params_RF

params_dict = random_search.best_params_
params_values = str(dict(zip( [key.replace('regressor__', '') for key in random_search.best_params_.keys()], random_search.best_params_.values())))
params_values

if 'correlation_selector' in pipeline.named_steps.keys():
    feature_select_meth ='correlation_selector {}'.format( pipeline.named_steps['correlation_selector'].get_params())
    print(feature_select_meth)
else: feature_select_meth = 'No feature selection'


if  rater_pred is not None:
    outcome_var = rater_pred.upper()
else:
    outcome_var = 'all'



correlation_selector {'threshold': 0.8}


In [1138]:

file_name_save = 'results_ML_simple_CV_RF_XGB.csv'
file_path_save = Path(data_derived, 'ML_results', file_name_save)

In [1139]:
if os.path.exists(file_path_save):
    df_result  = pd.read_csv(Path(data_derived, 'ML_results', file_name_save))
    #df_result.columns = cols_res
    print(df_result.columns)
    if 'Unnamed: 0' in df_result.columns:
        df_result = df_result.drop(columns= 'Unnamed: 0')
else:
    df_result = pd.DataFrame(columns= cols_res)
    

new_row = dict(zip(cols_res, [model_name, cross_val_strategy, params_values, mse, rmse, mae, r2,  y_col,outcome_var, feature_select_meth, num_features,thr_drop_missing]))

try: 
    if not (df_result.iloc[-1] == pd.Series(new_row)).all() :
        result_df = pd.concat([df_result, pd.DataFrame([new_row])], ignore_index=True)
        
    else: 
        print('ROW ALREADY EXISTS')
        result_df = df_result


except IndexError as e:
    if str(e) == 'single positional indexer is out-of-bounds':
        result_df = pd.DataFrame([new_row])

result_df


Index(['Unnamed: 0', 'Model Name', 'Cross Validation Type', 'Hyperparameters',
       'Mean Squared Error', 'Root Mean Squared Error', 'Mean Absolute Error',
       'R² Score', 'Outcome Variable', 'Input Data',
       'Feature Selection Method', 'Number of Features', 'Threshold Drop Row'],
      dtype='object')


,Model Name,Cross Validation Type,Hyperparameters,Mean Squared Error,Root Mean Squared Error,Mean Absolute Error,R² Score,Outcome Variable,Input Data,Feature Selection Method,Number of Features,Threshold Drop Row
0,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.5665,4.7504,3.7459,0.2925,snap_snaoddt_m_out,all,No feature selection,1026,50
1,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",29.9671,5.4742,4.3821,0.0572,snap_snaoddt_m_out,F,No feature selection,323,50
2,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",31.2867,5.5934,4.4474,0.0171,snap_snaoddt_m_out,T,No feature selection,207,50
3,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.1251,4.7037,3.7074,0.3062,snap_snaoddt_m_out,M,No feature selection,404,50
4,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.5913,4.7530,3.7425,0.2918,snap_snaoddt_m_out,all,correlation_selector {'threshold': 0.8},611,50
5,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.0846,4.6994,3.6951,0.3076,snap_snaoddt_m_out,M,correlation_selector {'threshold': 0.8},258,50
6,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",29.4567,5.4274,4.3311,0.0731,snap_snaoddt_m_out,F,correlation_selector {'threshold': 0.8},187,50
7,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",31.1534,5.5815,4.4088,0.0218,snap_snaoddt_m_out,T,correlation_selector {'threshold': 0.8},120,50
8,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.3987,4.7327,3.7335,0.2974,snap_snaoddt_m_out,all,No feature selection,627,20
9,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.1944,4.7110,3.7099,0.3040,snap_snaoddt_m_out,M,No feature selection,324,20


In [1140]:
nrows2drop = 1
df_result_reduced = result_df.iloc[:df_result.shape[0], :]
df_result_reduced

,Model Name,Cross Validation Type,Hyperparameters,Mean Squared Error,Root Mean Squared Error,Mean Absolute Error,R² Score,Outcome Variable,Input Data,Feature Selection Method,Number of Features,Threshold Drop Row
0,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.5665,4.7504,3.7459,0.2925,snap_snaoddt_m_out,all,No feature selection,1026,50
1,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",29.9671,5.4742,4.3821,0.0572,snap_snaoddt_m_out,F,No feature selection,323,50
2,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",31.2867,5.5934,4.4474,0.0171,snap_snaoddt_m_out,T,No feature selection,207,50
3,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.1251,4.7037,3.7074,0.3062,snap_snaoddt_m_out,M,No feature selection,404,50
4,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.5913,4.7530,3.7425,0.2918,snap_snaoddt_m_out,all,correlation_selector {'threshold': 0.8},611,50
5,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.0846,4.6994,3.6951,0.3076,snap_snaoddt_m_out,M,correlation_selector {'threshold': 0.8},258,50
6,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",29.4567,5.4274,4.3311,0.0731,snap_snaoddt_m_out,F,correlation_selector {'threshold': 0.8},187,50
7,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",31.1534,5.5815,4.4088,0.0218,snap_snaoddt_m_out,T,correlation_selector {'threshold': 0.8},120,50
8,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.3987,4.7327,3.7335,0.2974,snap_snaoddt_m_out,all,No feature selection,627,20
9,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.1944,4.7110,3.7099,0.3040,snap_snaoddt_m_out,M,No feature selection,324,20


In [ ]:
save = False
reduced = False
if save :
    print('save')
    if reduced:
        print('reduced')
        df_result_reduced.to_csv(file_path_save)
    else:
        print('full')
        result_df.to_csv(file_path_save)
verify   = pd.read_csv(file_path_save)
verify

save
full


,Unnamed: 0,Model Name,Cross Validation Type,Hyperparameters,Mean Squared Error,Root Mean Squared Error,Mean Absolute Error,R² Score,Outcome Variable,Input Data,Feature Selection Method,Number of Features,Threshold Drop Row
0,0,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.5665,4.7504,3.7459,0.2925,snap_snaoddt_m_out,all,No feature selection,1026,50
1,1,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",29.9671,5.4742,4.3821,0.0572,snap_snaoddt_m_out,F,No feature selection,323,50
2,2,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",31.2867,5.5934,4.4474,0.0171,snap_snaoddt_m_out,T,No feature selection,207,50
3,3,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.1251,4.7037,3.7074,0.3062,snap_snaoddt_m_out,M,No feature selection,404,50
4,4,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.5913,4.7530,3.7425,0.2918,snap_snaoddt_m_out,all,correlation_selector {'threshold': 0.8},611,50
5,5,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.0846,4.6994,3.6951,0.3076,snap_snaoddt_m_out,M,correlation_selector {'threshold': 0.8},258,50
6,6,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",29.4567,5.4274,4.3311,0.0731,snap_snaoddt_m_out,F,correlation_selector {'threshold': 0.8},187,50
7,7,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",31.1534,5.5815,4.4088,0.0218,snap_snaoddt_m_out,T,correlation_selector {'threshold': 0.8},120,50
8,8,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.3987,4.7327,3.7335,0.2974,snap_snaoddt_m_out,all,No feature selection,627,20
9,9,RandomForestRegressor,GroupKFold(n_splits=5),"{'n_estimators': 100, 'min_samples_split': 2, ...",22.1944,4.7110,3.7099,0.3040,snap_snaoddt_m_out,M,No feature selection,324,20
